In [103]:
import numpy as np
import matplotlib.pyplot as plt

In [104]:
%matplotlib notebook

# Generate Synthetic Data

In [105]:
numSyntheticSamples = 100000
syntheticDataRange = np.linspace( 0, np.pi*numSyntheticSamples/10.0, numSyntheticSamples)
syntheticData = np.sin( syntheticDataRange )

In [106]:
if syntheticData.ndim < 2:
    syntheticData = np.expand_dims(syntheticData, axis=1)

In [107]:
syntheticData.shape

(100000, 1)

In [108]:
plt.figure()
plt.plot(syntheticData[:500])

# Define hyperparameters

In [109]:
# of samples per sensor for the micro model [sliding window of ~2.5 hrs]
hParams = {}
hParams['windowSamples'] = 30
hParams['nSensors'] = 1
hParams['overlapPercentage'] = .99
hParams['advanceSamples'] = ( hParams['windowSamples'] - int( np.floor( hParams['windowSamples'] * hParams['overlapPercentage'] ) ))

# Split into train and test set (.25 test data)

In [110]:
def train_test_split (x, testDataRatio = .25, trainDataAtStart = True):
    assert x.ndim > 1
    if trainDataAtStart:
        splitIndex = int( ( 1.0 - testDataRatio) * x.shape[0] )    
        
        xTrain = x[ 0:splitIndex, :]
        xTest = x[ splitIndex:, :]
    else:
        splitIndex = int( testDataRatio * x.shape[0] )
        xTest = x[ 0:splitIndex, :]
        xTrain = x[ splitIndex:, :]
        
    return xTrain, xTest

In [111]:
trainSplit, testSplit = train_test_split( syntheticData )

In [112]:
syntheticData.shape

(100000, 1)

In [113]:
trainSplit.shape

(75000, 1)

In [114]:
testSplit.shape

(25000, 1)

# No augmentation / noise injection

# Normalize data ( 0 mean, unit standard deviation )

In [115]:
# find normalization statistics
trainMeans = np.mean(trainSplit, axis=0)
trainSTDevs = np.std(trainSplit, axis=0)
print(trainMeans); print(trainSTDevs)

# normalize [ in place / overwrite ]
normalizedTrainData = (trainSplit - trainMeans) / (trainSTDevs + .0001)
normalizedTestData = (testSplit - trainMeans) / (trainSTDevs + .0001)

[ -3.93309992e-07]
[ 0.70710323]


# Generate shuffled windows

In [116]:
def reshape_into_shuffled_data_windows ( x, windowSize, advanceSamples ):
    nWindows = int( np.floor( (x.shape[0] - windowSize)/(advanceSamples*1.0) ) )
    # shuffle indexes
    shuffledWindowInds = np.arange(nWindows)
    np.random.shuffle(shuffledWindowInds)    
        
    nSensors = x.shape[1]
    outputMatrix = np.zeros((nWindows, windowSize * nSensors))
    
    # update data matrix on a row by row basis (choosing shuffled windows per row)
    for iWindow in range(nWindows):
        startIndex = shuffledWindowInds[iWindow] * advanceSamples
        endIndex = startIndex + windowSize
        
        # flatten/interleave sensor values
        for iSensor in range(nSensors):
            outputMatrix[iWindow, iSensor::nSensors] = x[startIndex:endIndex, iSensor]
    
    return outputMatrix, shuffledWindowInds


In [117]:
trainMatrix, trainShuffledWindowInds = reshape_into_shuffled_data_windows(normalizedTrainData, hParams['windowSamples'], hParams['advanceSamples'])
testMatrix, testShuffledWindowInds = reshape_into_shuffled_data_windows(normalizedTestData, hParams['windowSamples'], hParams['advanceSamples'])

viz_flag = 1
if viz_flag:
    plt.figure()
    plt.plot(trainMatrix[200,:])


In [118]:
trainMatrix.shape

(74970, 30)

In [119]:
testMatrix.shape

(24970, 30)

# ML/DL Imports

In [120]:
from keras.models import Sequential, Model
from keras.layers import Dense
from keras import metrics
from keras import initializers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

# Model definition

In [121]:
hParams['inputOutputDimensionality'] = int( hParams['windowSamples'] * hParams['nSensors'] ) 
assert hParams['inputOutputDimensionality'] == trainMatrix.shape[1]

# Define model
model = Sequential()

In [122]:
model.add( Dense( 15, input_dim = hParams['inputOutputDimensionality'], activation = 'linear'))
model.add( Dense( 5, activation = 'sigmoid'))
model.add( Dense( 15, activation = 'sigmoid'))
model.add( Dense( hParams['inputOutputDimensionality'], activation = 'linear',))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 15)                465       
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 80        
_________________________________________________________________
dense_7 (Dense)              (None, 15)                90        
_________________________________________________________________
dense_8 (Dense)              (None, 30)                480       
Total params: 1,115
Trainable params: 1,115
Non-trainable params: 0
_________________________________________________________________


In [123]:
import nnViz
plt.figure()
nnViz.visualize_model(model)

In [124]:
model.compile(optimizer = 'adam', loss = 'mse')

In [36]:
early_stopping = EarlyStopping( monitor = 'val_loss', patience = 10)
checkpointer = ModelCheckpoint( filepath = 'synthetic_sin_weights_2.hdf5', verbose=1, save_best_only = True)

model.fit( trainMatrix,
           trainMatrix,
           batch_size = 256, epochs = 500,
           shuffle = True,
           callbacks = [early_stopping, checkpointer],
           validation_data = (testMatrix, testMatrix) )

RuntimeError: The model needs to be compiled before being used.

# Load best model weights

In [125]:
model.load_weights("synthetic_sin_weights_2.hdf5")
model.compile(optimizer = 'adam', loss = 'mse') # need to recompile model to be able to run prediction

# Plot raw vs predicted 

In [126]:
def windowed_predict(data, windowSize):    
    nWindows = int( data.size / (windowSize*1.0) )
    print('number of windows: ' + str(nWindows))
    predicted = np.zeros((data.shape[0], data.shape[1]))
    
    for iWindow in range(nWindows):
        dataStartIndex = int( iWindow * windowSize )
        dataEndIndex = dataStartIndex + windowSize
        
        predictedWindow = model.predict( np.transpose( data[dataStartIndex:dataEndIndex]) )
        predicted[dataStartIndex:dataEndIndex] = np.transpose(predictedWindow)
        
    return predicted

# Inject Anomalies & Predict

In [127]:
anomalySignal1 = np.exp(np.linspace(0, 1.5, 1000)) - 1
anomalySignal2 = np.cos(np.linspace(0,2*np.pi * 1, 1000))
anomalySignal3 = np.cos(np.linspace(0,2*np.pi * 10, 1000))

anomalySignal1 = np.expand_dims(anomalySignal1, axis=1)
anomalySignal2 = np.expand_dims(anomalySignal2, axis=1)
anomalySignal3 = np.expand_dims(anomalySignal3, axis=1)

plt.figure()
plt.plot(anomalySignal1)
plt.plot(anomalySignal2)
plt.plot(anomalySignal3)
plt.legend(['anomaly signal 1', 'anomaly signal 2', 'anomaly signal 3'])

In [128]:
startIndex = 0
endIndex = 1000
len(anomalySignal1)

1000

In [129]:
targetData = np.concatenate( ( normalizedTestData[startIndex:endIndex], anomalySignal1, \
                               normalizedTestData[startIndex:endIndex], anomalySignal2, \
                               normalizedTestData[startIndex:endIndex], anomalySignal3 ) )

anomalousInds_1 = np.arange(1000/hParams['windowSamples'], 2000/hParams['windowSamples'], dtype=int)
anomalousInds_2 = np.arange(3000/hParams['windowSamples'], 4000/hParams['windowSamples'], dtype=int)
anomalousInds_3 = np.arange(5000/hParams['windowSamples'], 6000/hParams['windowSamples'], dtype=int)

In [130]:
predictedData = windowed_predict ( targetData, hParams['inputOutputDimensionality'])
error = np.sqrt((targetData - predictedData)**2)

number of windows: 200


In [131]:
plt.figure(figsize=(10,10))

ax1 = plt.subplot2grid((4, 1), (0, 0), rowspan=3)
ax2 = plt.subplot2grid((4, 1), (3, 0), rowspan=1)

ax1.plot(targetData)
ax1.plot(predictedData, 'rx')
ax1.set_title('actual vs predicted')
ax1.legend(['actual', 'predicted'])

ax2.plot(error)
#ax2.set_ylim([0,5])
ax2.set_title('error over time -- avg error: ' + str(round(np.mean(error),4)))

## Remove last two layers [ focus on bottleneck activations ]

In [132]:
print(len(model.layers))

4


In [133]:
model.pop()
model.pop()

In [134]:
print(len(model.layers))

2


In [135]:
import nnViz
plt.figure()
nnViz.visualize_model(model)

In [136]:
model.compile(optimizer = 'adam', loss = 'mse')

#TODO : no overlaps

In [137]:
def windowed_predict_bottleneck_activation (data, windowSize, bottleneckSize):    
    nWindows = int( data.size / (windowSize*1.0) )
    print('number of windows: ' + str(nWindows))
    predicted = np.zeros((nWindows, bottleneckSize))
    
    #dataStartIndex = 0
    #advanceSamples = 1
    for iWindow in range(nWindows):
        #dataStartIndex += advanceSamples#int( iWindow * windowSize )
        dataStartIndex = int( iWindow * windowSize )
        dataEndIndex = dataStartIndex + windowSize
        
        predictedWindow = model.predict( np.transpose( data[dataStartIndex:dataEndIndex]) )
        predicted[iWindow, :] = predictedWindow[0]
        
        
    return predicted

In [138]:
bottleNeckSize = model.layers[-1].get_config()['units']
bottleneckActivations = windowed_predict_bottleneck_activation (targetData, hParams['inputOutputDimensionality'], bottleNeckSize)

number of windows: 200


In [139]:
from sklearn.manifold import TSNE

embeddedBottleneckActivations = TSNE(n_components = 2, perplexity = 10, learning_rate = 100, method='exact', verbose = 1).fit_transform(bottleneckActivations)

[t-SNE] Computing pairwise distances...
[t-SNE] Computed conditional probabilities for sample 200 / 200
[t-SNE] Mean sigma: 0.000284
[t-SNE] KL divergence after 100 iterations with early exaggeration: 10.234366
[t-SNE] Error after 750 iterations: 10.234366


In [140]:
plt.figure()
plt.plot(embeddedBottleneckActivations[:,0], embeddedBottleneckActivations[:,1], 'bx')
plt.plot(embeddedBottleneckActivations[anomalousInds_1,0], embeddedBottleneckActivations[anomalousInds_1,1], 'rx')
plt.plot(embeddedBottleneckActivations[anomalousInds_2,0], embeddedBottleneckActivations[anomalousInds_2,1], 'gx')
plt.plot(embeddedBottleneckActivations[anomalousInds_3,0], embeddedBottleneckActivations[anomalousInds_3,1], 'kx')


In [ ]:
plt.figure()
plt.plot(embeddedBottleneckActivations[:,0], embeddedBottleneckActivations[:,1], 'bx')
plt.plot(embeddedBottleneckActivations[anomalousInds_1,0], embeddedBottleneckActivations[anomalousInds_1,1], 'rx')
plt.plot(embeddedBottleneckActivations[anomalousInds_2,0], embeddedBottleneckActivations[anomalousInds_2,1], 'gx')
plt.plot(embeddedBottleneckActivations[anomalousInds_3,0], embeddedBottleneckActivations[anomalousInds_3,1], 'kx')


In [141]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
pca = PCA(n_components=2)
PCA_bottleneckActivations = pca.fit_transform(bottleneckActivations)

In [149]:
plt.figure()
plt.scatter(PCA_bottleneckActivations[:,0],PCA_bottleneckActivations[:,1], color='b')
plt.scatter(PCA_bottleneckActivations[anomalousInds_1,0],PCA_bottleneckActivations[anomalousInds_1,1], color='r')
plt.scatter(PCA_bottleneckActivations[anomalousInds_2,0],PCA_bottleneckActivations[anomalousInds_2,1], color='g')
plt.scatter(PCA_bottleneckActivations[anomalousInds_3,0],PCA_bottleneckActivations[anomalousInds_3,1], color='k')

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

In [143]:
import mpld3
from mpld3 import plugins, utils

In [59]:
targetData.shape

(6000, 1)

In [60]:
PCA_bottleneckActivations.shape

(200, 2)

In [61]:
range(len(firstSet))

range(0, 2000)

In [144]:
%matplotlib ipympl
# https://github.com/matplotlib/jupyter-matplotlib

In [145]:
import matplotlib.pyplot as plt
import numpy as np

In [146]:
fig = plt.figure(figsize=(10,10))

ax1 = plt.subplot(2,1,1)

ax1.plot( range(len(targetData)), targetData, 'x', picker = 1 )

ax2 = plt.subplot(2,1,2)
ax2.plot ( PCA_bottleneckActivations[:,0], PCA_bottleneckActivations[:,1], 'o', picker = 2 )

 index range:  3210, 3330
 index range:  5250, 5370
 index range:  5460, 5580
 index range:  5370, 5490
 index range:  5700, 5820
 index range:  960, 1080
 index range:  990, 1110
 index range:  1110, 1230
 index range:  1140, 1260
 index range:  1170, 1290
 index range:  1230, 1350
 index range:  1530, 1650


In [147]:
def updateRawPlot ( ind ):
    print(' index range:  ' + str(int((ind-2)*30)) + ', '+ str(int((ind+2)*30)))
    ax1.plot( list(range(int((ind-2)*30),int((ind+2)*30))), targetData[ int((ind-2)*30):int((ind+2)*30) ] )

In [148]:
#ax.scatter(list(range(100)),np.random.rand(100))#, 'o', picker=5)  # 5 points tolerance
#ax.plot(np.random.rand(100), 'o', picker=5)  # 5 points toleranc
def on_pick(event):
    if event.mouseevent.inaxes == ax2:          
        eventArtist = event.artist
        #xdata, ydata = eventArtist.get_data()
        ind = event.ind
        updateRawPlot( ind )
        #print('on pick line:', ind, np.array([xdata[ind], ydata[ind]]).T)
    
cid = fig.canvas.mpl_connect('pick_event', on_pick)
#plt.show()